In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
data_path = "/content/gdrive/My Drive/Project/Stocks/"

In [4]:
def get_all_files(data_path=data_path):
    all_files = []

    for file in os.listdir(data_path)[:100]:
        f = open(data_path+file)
        if len(f.readlines()) > 100: 
            all_files.append(data_path+file)
        f.close()
    
    return all_files

all_files = get_all_files()

In [5]:
n = len(all_files)

all_files = np.array(all_files, dtype=str)
np.random.shuffle(all_files)

training_data = all_files[:int(n*0.6)] 
validation_data = all_files[int(n*0.6):int(n*0.8)]
test_data = all_files[int(n*0.8):]

In [6]:
def transform_data(file, K):
    all_data = []

    f = open(file, "r")

    data = []
    for line in f.readlines()[-100:]:
        line = line.split(",")[1:-2]

        data.append(line)
        if len(data) == K+1:
            all_data.append((np.array(data[:-1], dtype=float), float(data[-1][-1])))
            data = data[1:]

    f.close()
    
    return all_data